In [6]:
import numpy as np
import matplotlib.pyplot as plt 
import random as rn

In [30]:
class particle: 
    def __init__(self,m,x,y,vx=0,vy=0): 
        """
        Defines the particle. Each particle has a mass, a position and momentum
        Inputs: 
            - m (float): mass of the particle 
            - x,y (float): x and y components respectively of the particle's position 
            - vx,vy (float): x and y components respectively of the particle's velocity
        """
        self.mass = m 
        self.position = (x,y)
        self.momentum = (self.mass*vx,self.mass*vy)
    
def evolve(self,position,momentum,mass,f_current,f_next,dt):
    """
    Evolves the particle's position and momentum using the leap frog method seen in class
    Inputs: 
        - f_current (array): The forces on the particle 
        - f_new (array): changed forces on the particle 
        - dt (float): time step in seconds 
    """
    x = position+(momentum/self.mass)*dt + 0.5*f_current*dt**2
    v = (momentum/mass) + 0.5*(f_current+f_next)*dt
    return x,v 

def calculate_dist(grid,pos):
    """
    Calculates the position between a grid point and a particle's position
    """
    return np.sqrt((grid[0]-pos[0])**2+(grid[1]-pos[1])**2)

In [31]:
class system: 
    def __init__(self,npart,size,init_mass,npart_specific=None):
        """
        Defines the total system of particles and puts them in a list. 
        Inputs: 
            - npast (int): total number of particles 
            - size (int): (x,y) of the size of the grid 
        """
        def initial_condition(size,npart,npart_specific=None): 
            """
            Depending on the grid size, generates random position on the grid. 
            Makes sure that the particles do not get generated on top of each other.
            Inputs:
                - size (array): size of grid. In the format of (x,y)
                - npart (int): total number of particles of the system
                - npar_specific (array): specific boundary condition pre-defined by the user
            """
            init_cond = []
            if npart_specific is None: 
                l = 0
            else: 
                l = len(npart_specific)
                for k in range(l):
                    pos = (npart_specific[k][0], npart_specific[k][1])
                    init_cond.append(pos)

            for p in range(npart-l):
                pos = (rn.random())*(size[0]-1), (rn.random())*(size[1]-1)
                
                init_cond.append(pos)
                #Small code block to make sure no two particles are generated on-top 
                #of each other. Although it is highly unlikely but just to make sure 
                j = 0 
                if p!=0:
                    while j<npart: 
                        if init_cond[-2][0] == init_cond[-1][0] and init_cond[-2][1] == init_cond[-1][1]:
                            x_in, y_in =  rn.random()*size[0], rn.random*size[1]
                            init_cond.pop()
                            init_cond.append([x_in,y_in])
                            j = -1
                        else: 
                            j += 1
                            
            return init_cond
        
        self.nparticles = npart
        init_cond = initial_condition(size,npart,npart_specific=npart_specific)
        self.particles = np.asarray([particle(m,x[0],x[1]) for m,x in zip(init_mass,init_cond)])   
        self.momenta = np.asarray([self.particles[i].momentum for i in range(npart)])
        self.pos = np.asarray([self.particles[i].position for i in range(npart)])
        self.masses = init_mass
        
    def evolve_system(self,acc,acc_new,dt,size,type_p = False):
        self.pos, self.momenta = evolve(self.pos,self.momenta,self.masses,acc,acc_new,dt)
        if type_p == True:
            self.pos = self.pos%(size-1)  

In [32]:
class grid: 
    def __init__(self,size,particleList):
        self.posP = particleList.pos
        self.intPos = np.round(self.posP)
        self.gridPos = self.grid_Maker(self.posP)
        self.mass = particleList.masses
        self.densities = self.density_assignment(self.posP,self.gridPos,self.mass,size)
    
    def grid_Maker(self,posP):
        """
        Function that generates the closest 4 gridpoints on the grid for each particle. 
        """
        gridPos = []
        #top right corner grid 
        gridPos = []
        gridPos.append(np.ceil(self.posP))  
        #top left corner grid
        gridPos.append(np.asarray([np.floor(self.posP[:,0]),np.ceil(self.posP[:,1])]).T) 
        #bottom right corner grid
        gridPos.append(np.asarray([np.ceil(self.posP[:,0]),np.floor(self.posP[:,1])]).T) 
        #bottom left corner grid 
        gridPos.append(np.floor(self.posP)) 
        
        return np.transpose(np.asarray(gridPos),axes=[1,0,-1])
    
    def updatePos(self,particleList):
        self.posP = particleList.pos
        self.intPos = np.round(self.posP)
        self.gridPos = self.grid_Maker(self.posP)
        self.mass = particleList.masses
        self.gridDensity = self.density(self.posP,self.gridPos,self.mass,size) 

    
        
    def density_assignment(self,posP,gridPos,mass,size):
        """
        Function that assigns the density based off the CIC method (cloud-in-cell) method
        """
        new_gridD = np.zeros((size[0],size[1]))
        
        for i in range(posP.shape[0]):
            for j in range(4):
                d = calculate_dist(gridPos[i,j],posP[i])
                new_gridD[int(gridPos[i,j,0]),int(gridPos[i,j,1])] += mass[i]*(1-d/np.sqrt(2))
        
        return new_gridD

In [34]:
npart = 2
size = (4,4)
init_mas = [rn.random() for t in range(npart)]
s = system(npart,size,init_mas,npart_specific=None)
g = grid(size,s)